Импорты

In [93]:
import warnings
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
warnings.filterwarnings('ignore')
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, BaggingRegressor, \
    GradientBoostingRegressor, StackingRegressor

Загружаем датасет

In [94]:
import pandas as pd

pokemon = pd.read_csv("../data/Pokemon_preprocessed.csv", index_col=0)
pokemon.info()
pokemon.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 799
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        800 non-null    object
 1   Total       800 non-null    int64 
 2   HP          800 non-null    int64 
 3   Attack      800 non-null    int64 
 4   Defense     800 non-null    int64 
 5   Sp. Atk     800 non-null    int64 
 6   Sp. Def     800 non-null    int64 
 7   Speed       800 non-null    int64 
 8   Generation  800 non-null    int64 
 9   Legendary   800 non-null    bool  
 10  t_Bug       800 non-null    int64 
 11  t_Dark      800 non-null    int64 
 12  t_Dragon    800 non-null    int64 
 13  t_Electric  800 non-null    int64 
 14  t_Fairy     800 non-null    int64 
 15  t_Fighting  800 non-null    int64 
 16  t_Fire      800 non-null    int64 
 17  t_Flying    800 non-null    int64 
 18  t_Ghost     800 non-null    int64 
 19  t_Grass     800 non-null    int64 
 20  t_Ground  

,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,t_Ghost,t_Grass,t_Ground,t_Ice,t_Normal,t_Poison,t_Psychic,t_Rock,t_Steel,t_Water
0,Bulbasaur,318,45,49,49,65,65,45,1,False,...,0,1,0,0,0,1,0,0,0,0
1,Ivysaur,405,60,62,63,80,80,60,1,False,...,0,1,0,0,0,1,0,0,0,0
2,Venusaur,525,80,82,83,100,100,80,1,False,...,0,1,0,0,0,1,0,0,0,0
3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,False,...,0,1,0,0,0,1,0,0,0,0
4,Charmander,309,39,52,43,60,50,65,1,False,...,0,0,0,0,0,0,0,0,0,0


# Классификация

Разбиваем данные на параметры входные и целевые.

In [95]:
X = pokemon.drop(["Legendary","Name"], axis=1).values
Y = pokemon.iloc[:, 9].values

In [96]:
print(X)

[[318  45  49 ...   0   0   0]
 [405  60  62 ...   0   0   0]
 [525  80  82 ...   0   0   0]
 ...
 [600  80 110 ...   0   0   0]
 [680  80 160 ...   0   0   0]
 [600  80 110 ...   0   0   1]]


In [97]:
print(Y)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True  True False False False  True  True  T

Разбиваем данные на обучающие и тестовые.

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [99]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Базовая модель

Обучим базовую модель и найдем гиперпараметры

In [100]:
%%time
parameters_clf = {'max_depth': np.arange(5,16,1)}
clf = DecisionTreeClassifier()
base = GridSearchCV(clf, parameters_clf).fit(X_train, y_train)
base.best_params_

Wall time: 143 ms


{'max_depth': 5}

In [101]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = base.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[143   2]
 [  7   8]]
              precision    recall  f1-score   support

       False       0.95      0.99      0.97       145
        True       0.80      0.53      0.64        15

    accuracy                           0.94       160
   macro avg       0.88      0.76      0.80       160
weighted avg       0.94      0.94      0.94       160



## Модель BaggingClassifier

In [102]:
%%time
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=base.best_params_['max_depth']))
model = GridSearchCV(bag, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 10.5 s


{'max_features': 23, 'n_estimators': 80}

In [103]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[142   3]
 [  5  10]]
              precision    recall  f1-score   support

       False       0.97      0.98      0.97       145
        True       0.77      0.67      0.71        15

    accuracy                           0.95       160
   macro avg       0.87      0.82      0.84       160
weighted avg       0.95      0.95      0.95       160



Наблюдаем заметные улучшения работы модели

## Модель GradientBoostingClassifier

In [104]:
%%time
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

gbc = GradientBoostingClassifier()
model = GridSearchCV(gbc, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 5.73 s


{'max_features': 23, 'n_estimators': 40}

In [105]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[142   3]
 [  6   9]]
              precision    recall  f1-score   support

       False       0.96      0.98      0.97       145
        True       0.75      0.60      0.67        15

    accuracy                           0.94       160
   macro avg       0.85      0.79      0.82       160
weighted avg       0.94      0.94      0.94       160



Продолжаем наблюдать за улучшением работы алгоритма

## Модель StackingClassifier

In [106]:
%%time
model = StackingClassifier(estimators=[('bag',bag), ('gbc',gbc)],
                           final_estimator=base).fit(X_train, y_train)

Wall time: 1.31 s


In [107]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[140   5]
 [  6   9]]
              precision    recall  f1-score   support

       False       0.96      0.97      0.96       145
        True       0.64      0.60      0.62        15

    accuracy                           0.93       160
   macro avg       0.80      0.78      0.79       160
weighted avg       0.93      0.93      0.93       160



## Модель CatBoostClassifier

In [108]:
from catboost import CatBoostClassifier, CatBoostRegressor

In [122]:
%%time
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train, y_train)
y_pred = cbc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0:	learn: 1.7699703	total: 10.4ms	remaining: 10.4s
1:	learn: 1.7546285	total: 18.5ms	remaining: 9.22s
2:	learn: 1.7348150	total: 27ms	remaining: 8.96s
3:	learn: 1.7112722	total: 36.6ms	remaining: 9.12s
4:	learn: 1.6933563	total: 45.2ms	remaining: 8.99s
5:	learn: 1.6767050	total: 53.2ms	remaining: 8.81s
6:	learn: 1.6583773	total: 66.3ms	remaining: 9.4s
7:	learn: 1.6452258	total: 77.9ms	remaining: 9.66s
8:	learn: 1.6342744	total: 85.6ms	remaining: 9.43s
9:	learn: 1.6238831	total: 93.3ms	remaining: 9.24s
10:	learn: 1.6163121	total: 101ms	remaining: 9.07s
11:	learn: 1.6035239	total: 109ms	remaining: 8.98s
12:	learn: 1.5940315	total: 117ms	remaining: 8.91s
13:	learn: 1.5842196	total: 126ms	remaining: 8.84s
14:	learn: 1.5767706	total: 133ms	remaining: 8.75s
15:	learn: 1.5708230	total: 141ms	remaining: 8.66s
16:	learn: 1.5613566	total: 149ms	remaining: 8.6s
17:	learn: 1.5508056	total: 157ms	remaining: 8.54s
18:	learn: 1.5417617	total: 164ms	remaining: 8.46s
19:	learn: 1.5349928	total: 172ms	r

160:	learn: 0.7259356	total: 1.67s	remaining: 8.68s
161:	learn: 0.7229275	total: 1.68s	remaining: 8.67s
162:	learn: 0.7187845	total: 1.69s	remaining: 8.65s
163:	learn: 0.7175121	total: 1.69s	remaining: 8.64s
164:	learn: 0.7157956	total: 1.7s	remaining: 8.62s
165:	learn: 0.7131357	total: 1.71s	remaining: 8.6s
166:	learn: 0.7103819	total: 1.72s	remaining: 8.58s
167:	learn: 0.7074263	total: 1.73s	remaining: 8.56s
168:	learn: 0.7028964	total: 1.74s	remaining: 8.54s
169:	learn: 0.6975658	total: 1.74s	remaining: 8.51s
170:	learn: 0.6942104	total: 1.75s	remaining: 8.49s
171:	learn: 0.6911636	total: 1.76s	remaining: 8.47s
172:	learn: 0.6872655	total: 1.77s	remaining: 8.45s
173:	learn: 0.6839107	total: 1.77s	remaining: 8.43s
174:	learn: 0.6801032	total: 1.78s	remaining: 8.4s
175:	learn: 0.6757576	total: 1.79s	remaining: 8.38s
176:	learn: 0.6721082	total: 1.8s	remaining: 8.37s
177:	learn: 0.6687481	total: 1.81s	remaining: 8.34s
178:	learn: 0.6676367	total: 1.81s	remaining: 8.32s
179:	learn: 0.66

336:	learn: 0.3572320	total: 3.28s	remaining: 6.46s
337:	learn: 0.3560700	total: 3.29s	remaining: 6.45s
338:	learn: 0.3551510	total: 3.3s	remaining: 6.44s
339:	learn: 0.3544726	total: 3.31s	remaining: 6.43s
340:	learn: 0.3528893	total: 3.32s	remaining: 6.42s
341:	learn: 0.3513486	total: 3.33s	remaining: 6.41s
342:	learn: 0.3499371	total: 3.34s	remaining: 6.39s
343:	learn: 0.3487674	total: 3.35s	remaining: 6.39s
344:	learn: 0.3473509	total: 3.36s	remaining: 6.38s
345:	learn: 0.3463771	total: 3.37s	remaining: 6.37s
346:	learn: 0.3451295	total: 3.38s	remaining: 6.35s
347:	learn: 0.3434602	total: 3.38s	remaining: 6.34s
348:	learn: 0.3422451	total: 3.39s	remaining: 6.33s
349:	learn: 0.3409469	total: 3.4s	remaining: 6.32s
350:	learn: 0.3398968	total: 3.41s	remaining: 6.3s
351:	learn: 0.3388813	total: 3.42s	remaining: 6.29s
352:	learn: 0.3383852	total: 3.43s	remaining: 6.28s
353:	learn: 0.3371163	total: 3.43s	remaining: 6.27s
354:	learn: 0.3357651	total: 3.44s	remaining: 6.25s
355:	learn: 0.3

507:	learn: 0.2156581	total: 5.06s	remaining: 4.9s
508:	learn: 0.2149370	total: 5.08s	remaining: 4.9s
509:	learn: 0.2146309	total: 5.09s	remaining: 4.89s
510:	learn: 0.2142725	total: 5.1s	remaining: 4.88s
511:	learn: 0.2135097	total: 5.11s	remaining: 4.87s
512:	learn: 0.2130601	total: 5.12s	remaining: 4.86s
513:	learn: 0.2124832	total: 5.13s	remaining: 4.85s
514:	learn: 0.2122020	total: 5.14s	remaining: 4.84s
515:	learn: 0.2115968	total: 5.15s	remaining: 4.83s
516:	learn: 0.2111278	total: 5.16s	remaining: 4.82s
517:	learn: 0.2104922	total: 5.17s	remaining: 4.81s
518:	learn: 0.2100631	total: 5.18s	remaining: 4.8s
519:	learn: 0.2096854	total: 5.19s	remaining: 4.79s
520:	learn: 0.2090485	total: 5.2s	remaining: 4.78s
521:	learn: 0.2086205	total: 5.21s	remaining: 4.77s
522:	learn: 0.2080706	total: 5.22s	remaining: 4.76s
523:	learn: 0.2077745	total: 5.23s	remaining: 4.75s
524:	learn: 0.2070325	total: 5.24s	remaining: 4.74s
525:	learn: 0.2064097	total: 5.25s	remaining: 4.73s
526:	learn: 0.206

681:	learn: 0.1481356	total: 6.87s	remaining: 3.2s
682:	learn: 0.1476730	total: 6.88s	remaining: 3.19s
683:	learn: 0.1473699	total: 6.89s	remaining: 3.18s
684:	learn: 0.1470779	total: 6.9s	remaining: 3.17s
685:	learn: 0.1466445	total: 6.91s	remaining: 3.16s
686:	learn: 0.1463636	total: 6.92s	remaining: 3.15s
687:	learn: 0.1460497	total: 6.93s	remaining: 3.14s
688:	learn: 0.1457375	total: 6.94s	remaining: 3.13s
689:	learn: 0.1454339	total: 6.95s	remaining: 3.12s
690:	learn: 0.1451923	total: 6.96s	remaining: 3.11s
691:	learn: 0.1449039	total: 6.97s	remaining: 3.1s
692:	learn: 0.1446569	total: 6.99s	remaining: 3.09s
693:	learn: 0.1444426	total: 7s	remaining: 3.08s
694:	learn: 0.1440727	total: 7s	remaining: 3.07s
695:	learn: 0.1439174	total: 7.01s	remaining: 3.06s
696:	learn: 0.1435972	total: 7.02s	remaining: 3.05s
697:	learn: 0.1434185	total: 7.03s	remaining: 3.04s
698:	learn: 0.1432575	total: 7.04s	remaining: 3.03s
699:	learn: 0.1429103	total: 7.05s	remaining: 3.02s
700:	learn: 0.1427399

847:	learn: 0.1118120	total: 8.51s	remaining: 1.52s
848:	learn: 0.1115877	total: 8.52s	remaining: 1.51s
849:	learn: 0.1114312	total: 8.53s	remaining: 1.5s
850:	learn: 0.1112174	total: 8.53s	remaining: 1.49s
851:	learn: 0.1111314	total: 8.54s	remaining: 1.48s
852:	learn: 0.1110162	total: 8.55s	remaining: 1.47s
853:	learn: 0.1108889	total: 8.56s	remaining: 1.46s
854:	learn: 0.1107319	total: 8.57s	remaining: 1.45s
855:	learn: 0.1105510	total: 8.59s	remaining: 1.44s
856:	learn: 0.1104799	total: 8.59s	remaining: 1.43s
857:	learn: 0.1103129	total: 8.61s	remaining: 1.42s
858:	learn: 0.1101189	total: 8.61s	remaining: 1.41s
859:	learn: 0.1099998	total: 8.62s	remaining: 1.4s
860:	learn: 0.1097823	total: 8.63s	remaining: 1.39s
861:	learn: 0.1097098	total: 8.64s	remaining: 1.38s
862:	learn: 0.1095803	total: 8.65s	remaining: 1.37s
863:	learn: 0.1094112	total: 8.66s	remaining: 1.36s
864:	learn: 0.1092534	total: 8.67s	remaining: 1.35s
865:	learn: 0.1091683	total: 8.68s	remaining: 1.34s
866:	learn: 0.

#  Регрессия

Создаем функцию для вывода оценок моделей

In [110]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def print_estimation(estimator, y_train, y_pred):
    print("R^2 = ", estimator.score(X, Y))
    print("MSE = ", mean_squared_error(y_train, y_pred))
    print("RMSE = ", mean_squared_error(y_train, y_pred, squared=False))
    print("MAE = ", mean_absolute_error(y_train, y_pred))

Разбиваем данные на параметры входные и целевые

In [111]:
X = pokemon.drop(["Generation","Name"], axis=1).values
Y = pokemon.iloc[:, 8].values

In [112]:
print(X)
print(Y)

[[318 45 49 ... 0 0 0]
 [405 60 62 ... 0 0 0]
 [525 80 82 ... 0 0 0]
 ...
 [600 80 110 ... 0 0 0]
 [680 80 160 ... 0 0 0]
 [600 80 110 ... 0 0 1]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3

Разбиваем данные на обучающие и тестовые

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

##  Базовая модель

Обучим базовую модель и найдем гиперпараметры

In [114]:
%%time
parameters_dtr = {'max_depth': np.arange(5,16,1)}
dtr = DecisionTreeRegressor().fit(X_train, y_train)
dtr_base = GridSearchCV(dtr, parameters_dtr).fit(X_train, y_train)
print(dtr_base.best_params_)
print_estimation(dtr_base, y_test, dtr_base.predict(X_test))

{'max_depth': 5}
R^2 =  0.17393699021378528
MSE =  2.719673476038141
RMSE =  1.649143255159521
MAE =  1.402377053619035
Wall time: 297 ms


## Модель BaggingRegressor

In [115]:
%%time
br = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=dtr_base.best_params_['max_depth']))
model = GridSearchCV(br, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 12 s


{'max_features': 23, 'n_estimators': 40}

In [116]:
print_estimation(model, y_test, dtr_base.predict(X_test))

R^2 =  0.23686942535260502
MSE =  2.719673476038141
RMSE =  1.649143255159521
MAE =  1.402377053619035


## Модель GradientBoostingRegressor

In [117]:
%%time
gbr = GradientBoostingRegressor()
model = GridSearchCV(gbr, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 3.67 s


{'max_features': 3, 'n_estimators': 80}

In [118]:
print_estimation(model, y_test, dtr_base.predict(X_test))

R^2 =  0.33248517851392145
MSE =  2.719673476038141
RMSE =  1.649143255159521
MAE =  1.402377053619035


## Модель StackingRegressor

In [119]:
%%time
model = StackingRegressor(estimators=[('br',br), ('gbr',gbr)],
                           final_estimator=dtr_base).fit(X_train, y_train)

Wall time: 1.14 s


In [120]:
print_estimation(model, y_test, dtr_base.predict(X_test))

R^2 =  0.18255668735208908
MSE =  2.719673476038141
RMSE =  1.649143255159521
MAE =  1.402377053619035


## Модель CatBoostRegressor

In [123]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)
print_estimation(cbr, y_test, dtr_base.predict(X_test))

0:	learn: 1.6338810	total: 2.28ms	remaining: 2.28s
1:	learn: 1.6194900	total: 5.65ms	remaining: 2.82s
2:	learn: 1.6053054	total: 9.79ms	remaining: 3.25s
3:	learn: 1.5885960	total: 12.2ms	remaining: 3.04s
4:	learn: 1.5744588	total: 15ms	remaining: 2.99s
5:	learn: 1.5652875	total: 17.5ms	remaining: 2.9s
6:	learn: 1.5537869	total: 19.9ms	remaining: 2.82s
7:	learn: 1.5445728	total: 22.7ms	remaining: 2.81s
8:	learn: 1.5386663	total: 25.2ms	remaining: 2.77s
9:	learn: 1.5329418	total: 27.7ms	remaining: 2.74s
10:	learn: 1.5236059	total: 30.2ms	remaining: 2.71s
11:	learn: 1.5139806	total: 32.7ms	remaining: 2.69s
12:	learn: 1.5067952	total: 35.2ms	remaining: 2.67s
13:	learn: 1.5013084	total: 37.7ms	remaining: 2.66s
14:	learn: 1.4947357	total: 40.7ms	remaining: 2.67s
15:	learn: 1.4884269	total: 43ms	remaining: 2.64s
16:	learn: 1.4795156	total: 45.7ms	remaining: 2.64s
17:	learn: 1.4725530	total: 48.3ms	remaining: 2.63s
18:	learn: 1.4621883	total: 51ms	remaining: 2.63s
19:	learn: 1.4537504	total: 5

162:	learn: 0.6970315	total: 504ms	remaining: 2.59s
163:	learn: 0.6921686	total: 509ms	remaining: 2.6s
164:	learn: 0.6882793	total: 513ms	remaining: 2.59s
165:	learn: 0.6843286	total: 518ms	remaining: 2.6s
166:	learn: 0.6838819	total: 521ms	remaining: 2.6s
167:	learn: 0.6835497	total: 524ms	remaining: 2.59s
168:	learn: 0.6813116	total: 527ms	remaining: 2.59s
169:	learn: 0.6781850	total: 530ms	remaining: 2.59s
170:	learn: 0.6745805	total: 535ms	remaining: 2.59s
171:	learn: 0.6712143	total: 540ms	remaining: 2.6s
172:	learn: 0.6665527	total: 543ms	remaining: 2.59s
173:	learn: 0.6632277	total: 546ms	remaining: 2.59s
174:	learn: 0.6616287	total: 549ms	remaining: 2.59s
175:	learn: 0.6578039	total: 552ms	remaining: 2.58s
176:	learn: 0.6546618	total: 556ms	remaining: 2.58s
177:	learn: 0.6543810	total: 560ms	remaining: 2.58s
178:	learn: 0.6525684	total: 563ms	remaining: 2.58s
179:	learn: 0.6507418	total: 566ms	remaining: 2.58s
180:	learn: 0.6480951	total: 571ms	remaining: 2.58s
181:	learn: 0.64

349:	learn: 0.3503193	total: 1.17s	remaining: 2.18s
350:	learn: 0.3490842	total: 1.18s	remaining: 2.18s
351:	learn: 0.3483831	total: 1.19s	remaining: 2.18s
352:	learn: 0.3476571	total: 1.19s	remaining: 2.18s
353:	learn: 0.3463111	total: 1.2s	remaining: 2.18s
354:	learn: 0.3440742	total: 1.2s	remaining: 2.18s
355:	learn: 0.3428281	total: 1.2s	remaining: 2.17s
356:	learn: 0.3415789	total: 1.21s	remaining: 2.17s
357:	learn: 0.3408525	total: 1.21s	remaining: 2.17s
358:	learn: 0.3391573	total: 1.21s	remaining: 2.17s
359:	learn: 0.3386671	total: 1.22s	remaining: 2.16s
360:	learn: 0.3379647	total: 1.22s	remaining: 2.16s
361:	learn: 0.3368826	total: 1.22s	remaining: 2.15s
362:	learn: 0.3365706	total: 1.23s	remaining: 2.15s
363:	learn: 0.3350750	total: 1.23s	remaining: 2.15s
364:	learn: 0.3337281	total: 1.23s	remaining: 2.14s
365:	learn: 0.3325469	total: 1.23s	remaining: 2.14s
366:	learn: 0.3319249	total: 1.24s	remaining: 2.13s
367:	learn: 0.3299213	total: 1.24s	remaining: 2.13s
368:	learn: 0.3

510:	learn: 0.2144363	total: 1.68s	remaining: 1.61s
511:	learn: 0.2135352	total: 1.68s	remaining: 1.6s
512:	learn: 0.2132207	total: 1.68s	remaining: 1.6s
513:	learn: 0.2125200	total: 1.69s	remaining: 1.59s
514:	learn: 0.2125043	total: 1.69s	remaining: 1.59s
515:	learn: 0.2119496	total: 1.69s	remaining: 1.59s
516:	learn: 0.2115612	total: 1.7s	remaining: 1.58s
517:	learn: 0.2111709	total: 1.7s	remaining: 1.58s
518:	learn: 0.2103920	total: 1.7s	remaining: 1.58s
519:	learn: 0.2097582	total: 1.71s	remaining: 1.57s
520:	learn: 0.2094467	total: 1.71s	remaining: 1.57s
521:	learn: 0.2092115	total: 1.71s	remaining: 1.57s
522:	learn: 0.2085395	total: 1.71s	remaining: 1.56s
523:	learn: 0.2081730	total: 1.72s	remaining: 1.56s
524:	learn: 0.2078150	total: 1.72s	remaining: 1.55s
525:	learn: 0.2069992	total: 1.72s	remaining: 1.55s
526:	learn: 0.2065862	total: 1.72s	remaining: 1.55s
527:	learn: 0.2059557	total: 1.73s	remaining: 1.54s
528:	learn: 0.2053522	total: 1.73s	remaining: 1.54s
529:	learn: 0.204

694:	learn: 0.1349466	total: 2.19s	remaining: 960ms
695:	learn: 0.1346328	total: 2.19s	remaining: 957ms
696:	learn: 0.1338825	total: 2.19s	remaining: 953ms
697:	learn: 0.1336504	total: 2.19s	remaining: 950ms
698:	learn: 0.1333670	total: 2.2s	remaining: 947ms
699:	learn: 0.1333266	total: 2.2s	remaining: 944ms
700:	learn: 0.1330232	total: 2.2s	remaining: 940ms
701:	learn: 0.1328460	total: 2.21s	remaining: 938ms
702:	learn: 0.1324046	total: 2.21s	remaining: 935ms
703:	learn: 0.1322135	total: 2.22s	remaining: 932ms
704:	learn: 0.1317484	total: 2.22s	remaining: 929ms
705:	learn: 0.1313464	total: 2.22s	remaining: 925ms
706:	learn: 0.1307856	total: 2.22s	remaining: 922ms
707:	learn: 0.1304989	total: 2.23s	remaining: 919ms
708:	learn: 0.1302561	total: 2.23s	remaining: 915ms
709:	learn: 0.1300262	total: 2.23s	remaining: 912ms
710:	learn: 0.1295210	total: 2.23s	remaining: 908ms
711:	learn: 0.1295068	total: 2.24s	remaining: 905ms
712:	learn: 0.1291601	total: 2.24s	remaining: 901ms
713:	learn: 0.1

870:	learn: 0.0926593	total: 2.69s	remaining: 399ms
871:	learn: 0.0924325	total: 2.7s	remaining: 396ms
872:	learn: 0.0921350	total: 2.7s	remaining: 393ms
873:	learn: 0.0917881	total: 2.7s	remaining: 390ms
874:	learn: 0.0916547	total: 2.71s	remaining: 386ms
875:	learn: 0.0913674	total: 2.71s	remaining: 384ms
876:	learn: 0.0910259	total: 2.71s	remaining: 380ms
877:	learn: 0.0908825	total: 2.71s	remaining: 377ms
878:	learn: 0.0905989	total: 2.72s	remaining: 374ms
879:	learn: 0.0901696	total: 2.72s	remaining: 371ms
880:	learn: 0.0900525	total: 2.73s	remaining: 368ms
881:	learn: 0.0899149	total: 2.73s	remaining: 365ms
882:	learn: 0.0896793	total: 2.73s	remaining: 362ms
883:	learn: 0.0896765	total: 2.73s	remaining: 359ms
884:	learn: 0.0896194	total: 2.73s	remaining: 355ms
885:	learn: 0.0893428	total: 2.74s	remaining: 352ms
886:	learn: 0.0889185	total: 2.74s	remaining: 349ms
887:	learn: 0.0888160	total: 2.74s	remaining: 346ms
888:	learn: 0.0886369	total: 2.75s	remaining: 343ms
889:	learn: 0.0

Данная модель обучилась намного лучше, чем предыдущие.

## Вывод

С помощью алгоритмов композиций, а также стекинга мы можем повышать качество моделей. Модели из библиотеки catboost выделяются своим качеством и скоростью обучения.